# Objective
Speed up my job search by automating searching through the websites I frequently use.

In [1]:
import numpy as np
import pandas as pd
import os
import datetime
import requests
import pprint
import re
from bs4 import BeautifulSoup

# Preparation

In [2]:
# Make root folder the current working directory
os.chdir('..')
# Folders
folder_settings = './data/'
folder_raw = './data/raw/'
folder_interim = './data/interim/'
folder_clean = './data/processed/'

In [3]:
# Create empty arrays per datapoint
job_titles = []
job_links = []
job_locations = []
job_companies = []
job_post_dates = []
job_source = []

In [4]:
date_today = datetime.date.today().strftime("%Y-%m-%d")

# Scrape websites

## Nextbillion

In [5]:
urls = ['https://nextbillion.net/jobs/?jobs-page=1', 'https://nextbillion.net/jobs/?jobs-page=2',  'https://nextbillion.net/jobs/?jobs-page=2']

In [6]:
# Use loop to extract required data per 
for url in urls:
    soup_page = requests.get(url)
    soup_page = BeautifulSoup(soup_page.content, 'html.parser')
    # grabs each job
    jobs = soup_page.findAll("li", {"class":"clearfix"})
    for job in jobs:
        job_title = job.h3.text
        job_titles.append(job_title)
        job_location = job.findAll("dd")[1].text
        job_locations.append(job_location)
        job_company = job.findAll("dd")[0].text
        job_companies.append(job_company)
        job_post_date = job.find("div", {"class":"event-date jobs-date"}).text
        job_post_dates.append(job_post_date)
        job_link = job.h3.a.get('href')
        job_links.append(job_link)
        job_source.append('nextbillion')

## Findevgateway

In [7]:
urls = 'https://www.findevgateway.org/jobs?job_type=All&regions=&countries=&remote=All&f%5B0%5D=job_remote%3A4111&f%5B1%5D=job_type%3A3906'

In [8]:
soup_page = requests.get(urls)
soup_page = BeautifulSoup(soup_page.content, 'html.parser')

In [9]:
for ad_n in list(range(0,7)):
    job_title = soup_page.findAll('a', {"href": re.compile('jobs-internships')})[ad_n].text
    job_titles.append(job_title)
    job_company = soup_page.findAll('a', {"href": re.compile('/organization/')})[ad_n].text
    job_companies.append(job_company)
    job_post_date = soup_page.findAll("span", {"class":"date"})[ad_n].text
    job_post_dates.append(job_post_date)
    job_location = soup_page.find_all(text=re.compile('Location'))[ad_n]
    job_locations.append(job_location)
    job_links.append('TBC')
    job_source.append('Findevgateway')

job_post_date = soup_page.findAll("span", {"class":"date"})[1].text.split(':')[1]
job_company = soup_page.findAll('a', {"href": re.compile('/organization/')})
job_location = soup_page.findAll("div", class_="postmeta")[1].text.split(':')[1]

soup_page.find_all(text=re.compile('Location'))

soup_page.findAll('a', {"href": re.compile('/organization/')}

job_location = soup_page.findAll("div", class_="postmeta")
#[ad_n].text
job_location

soup_page.findAll('a', {"href": re.compile('jobs-internships')})[3].text

soup_page.find_all(text=re.compile('Location'))[ad_n]

for ad in list(range(1,5)):
    job_post_date = soup_page.findAll("span", {"class":"date"})[item].text.split(':')[1]
    print(job_post_date)
    job_company = soup_page.findAll("div", class_="postmeta")[item].text
    print(job_company)
    job_location = soup_page.findAll("div", class_="postmeta")[item].text.split(':')[1]

qqq

job_company

soup_page.findAll("div", class_="postmeta")[1].text.split(':')[1]

qqq

soup_page.findAll("a", href="/organization/angaza")


soup_page.findAll('a', {"href": re.compile('/organization/')})

soup_page.findAll('div', {"href": "/organization/angaza"})

re.compile('/')

import re
soup_page.findAll('div', {"id": re.compile('/')})

soup_page.findAll('div', {"postmeta": lambda x: x and x.startswith('/*')})

for item in soup.findAll('a', href=: lambda x: x and x.startswith('/org')}):

soup_page.findAll("div", class_="postmeta")

soup_page.findAll("div", {"class":"meta"})

soup_page.findAll('a', href=True)

text = html.read()
plaintext = text.decode('utf8')

# Create dataframe

In [10]:
# create dataframe based on extracted data
jobs_df = pd.DataFrame({'job_title': job_titles,
                        'company': job_companies,
                        'location': job_locations,
                        'posting_date': job_post_dates,
                        'source': job_source,
                        'weblink': job_links})

In [11]:
jobs_df['job_title'].count()

31

# Filter relevant jobs

In [12]:
jobs_filtered_df = jobs_df.copy()

In [13]:
# make job titles lower case to prevent 
jobs_filtered_df['job_title'] = jobs_filtered_df['job_title'].str.lower()

In [14]:
filter_criteria = 'manager|data science|data scientist|analytics'

In [15]:
jobs_filtered_df = jobs_filtered_df[jobs_filtered_df['job_title'].str.contains(filter_criteria)]

# Export

In [16]:
# Export complete dataframe
filename = date_today + ' - Job search social data scientist - UNFILTERED.csv'
jobs_df.to_csv(folder_interim + filename)

In [18]:
# Export filtered data
filename = date_today + ' - Job search social data scientist - FILTERED.csv'
jobs_filtered_df.to_csv(folder_clean + filename)